In [1]:
import wrds
import pandas as pd
import numpy as np
from datetime import datetime
import pandas_datareader.data as web
from tqdm.notebook import tqdm
import os
import pyarrow as pa
import pyarrow.parquet as pq

# ─── CONFIGURATION ──────────────────────────────────────────────
YEARS        = range(2021, 2024)      # years to fetch
NUM_SECIDS   = 2000                   # number of SECIDs to sample
CHUNK_SIZE   = 200                    # SECIDs per chunk
COLUMNS_OP   = [
    'date','secid','strike_price','best_bid','best_offer',
    'impl_volatility','delta','gamma','vega','theta','exdate','cp_flag'
]
OUTFILE      = 'option_data_2021-2023_2000secids.parquet'
# ─────────────────────────────────────────────────────────────────

def chunk_list(lst, size):
    for i in range(0, len(lst), size):
        yield lst[i:i + size]

# 1) Sample SECIDs
db = wrds.Connection()
secid_df = db.raw_sql("SELECT DISTINCT secid FROM optionm.opprcd2023")
all_secids = secid_df['secid'].tolist()
np.random.seed(42)
sampled = np.random.choice(all_secids, NUM_SECIDS, replace=False)
db.close()

# 2) Pre-load risk-free rates
treasury = (
    web.DataReader('DGS3MO', 'fred', datetime(2021,1,1), datetime(2023,12,31))
      .rename(columns={'DGS3MO':'risk_free_rate'})
      .reset_index()
)
treasury['risk_free_rate'] /= 100
treasury['date'] = pd.to_datetime(treasury['DATE'])
treasury = treasury[['date','risk_free_rate']]

# 3) Stream fetch & write to Parquet (options + underlying)
writer = None
db     = wrds.Connection()

for year in YEARS:
    for chunk in tqdm(list(chunk_list(sampled, CHUNK_SIZE)), desc=f"Year {year}"):
        secids_sql = ','.join(f"'{s}'" for s in chunk)

        # Fetch option data
        sql_op = f"""
            SELECT {', '.join(COLUMNS_OP)}
            FROM optionm.opprcd{year}
            WHERE secid IN ({secids_sql})
        """
        df_op = db.raw_sql(sql_op)
        if df_op.empty:
            continue
        df_op.dropna(inplace=True)
        df_op['date']   = pd.to_datetime(df_op['date'])
        df_op['exdate'] = pd.to_datetime(df_op['exdate'])
        df_op['mid_price'] = (df_op['best_bid'] + df_op['best_offer']) / 2
        df_op = df_op.merge(treasury, on='date', how='left').ffill()

        # Fetch underlying price data
        sql_und = f"""
            SELECT secid, date, low, high, close, open
            FROM optionm.secprd{year}
            WHERE secid IN ({secids_sql})
        """
        df_und = db.raw_sql(sql_und)
        if df_und.empty:
            continue
        df_und['date'] = pd.to_datetime(df_und['date'])
        df_und['underlying_price'] = df_und[['low','high','close','open']].mean(axis=1)
        df_und = df_und[['secid','date','underlying_price']]

        # Merge options + underlying
        df = pd.merge(df_op, df_und, on=['secid','date'], how='inner')
        df.dropna(subset=['underlying_price'], inplace=True)

        # Write to Parquet
        table = pa.Table.from_pandas(df, preserve_index=False)
        if writer is None:
            writer = pq.ParquetWriter(OUTFILE, table.schema)
        writer.write_table(table)

db.close()
writer.close()

# 4) Report file size
size_bytes = os.path.getsize(OUTFILE)
for unit in ['B','KB','MB','GB']:
    if size_bytes < 1024:
        break
    size_bytes /= 1024
print(f"Final Parquet size: {size_bytes:.2f} {unit}")


WRDS recommends setting up a .pgpass file.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done
WRDS recommends setting up a .pgpass file.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


Year 2021:   0%|          | 0/10 [00:00<?, ?it/s]

Year 2022:   0%|          | 0/10 [00:00<?, ?it/s]

Year 2023:   0%|          | 0/10 [00:00<?, ?it/s]

Final Parquet size: 9.47 GB
